<a href="https://colab.research.google.com/github/dawid-gulczynski/machine-learning/blob/main/basics/03_feature_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import sklearn

sklearn.__version__

'1.2.2'

Tworzę funkcję, która z portalu Stooq pobiera 5 ostatnich notowań akcji spółki Amazon


In [2]:
def fetch_financial_data(company='AMZN'):
    """
    This function fetches stock market quotations.
    """
    import pandas_datareader.data as web
    return web.DataReader(name=company, data_source='stooq')

df_raw = fetch_financial_data()
df_raw.head()

,Open,High,Low,Close,Volume
Date,,,,,
2024-04-17,184.310,184.57,179.82,181.28,31359673
2024-04-16,183.270,184.83,182.26,183.32,32891265
2024-04-15,187.425,188.69,183.00,183.62,48052395
2024-04-12,187.720,188.38,185.08,186.13,38608849
2024-04-11,186.740,189.77,185.51,189.05,40020742


In [3]:
df = df_raw.copy()
df = df[:5]
#wycinamy tylko 5 pierwszych wierszy
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5 entries, 2024-04-17 to 2024-04-11
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    5 non-null      float64
 1   High    5 non-null      float64
 2   Low     5 non-null      float64
 3   Close   5 non-null      float64
 4   Volume  5 non-null      int64  
dtypes: float64(4), int64(1)
memory usage: 240.0 bytes


Generowanie nowych zmiennych


In [4]:
df['day'] = df.index.day
df['month'] = df.index.month
df['year'] = df.index.year
df

,Open,High,Low,Close,Volume,day,month,year
Date,,,,,,,,
2024-04-17,184.310,184.57,179.82,181.28,31359673,17,4,2024
2024-04-16,183.270,184.83,182.26,183.32,32891265,16,4,2024
2024-04-15,187.425,188.69,183.00,183.62,48052395,15,4,2024
2024-04-12,187.720,188.38,185.08,186.13,38608849,12,4,2024
2024-04-11,186.740,189.77,185.51,189.05,40020742,11,4,2024


Dyskretyzacja zmiennej ciągłej

In [5]:
df = pd.DataFrame(data={'height': [175., 178.5, 185., 191., 184.5, 183., 168.]})
df

,height
0,175.0
1,178.5
2,185.0
3,191.0
4,184.5
5,183.0
6,168.0


In [6]:
df['height-cat'] = pd.cut(x=df.height, bins=3)
df

,height,height-cat
0,175.0,"(167.977, 175.667]"
1,178.5,"(175.667, 183.333]"
2,185.0,"(183.333, 191.0]"
3,191.0,"(183.333, 191.0]"
4,184.5,"(183.333, 191.0]"
5,183.0,"(175.667, 183.333]"
6,168.0,"(167.977, 175.667]"


In [7]:
df['height-cat'] = pd.cut(x=df.height, bins=(160,175,180,195))
df

,height,height-cat
0,175.0,"(160, 175]"
1,178.5,"(175, 180]"
2,185.0,"(180, 195]"
3,191.0,"(180, 195]"
4,184.5,"(180, 195]"
5,183.0,"(180, 195]"
6,168.0,"(160, 175]"


Dodajemy wypisywanie kategorii zamiast przedziału

In [8]:
df['height-cat'] = pd.cut(x=df.height, bins=(160,175,180,195), labels=['small', 'medium', 'high'])
df

,height,height-cat
0,175.0,small
1,178.5,medium
2,185.0,high
3,191.0,high
4,184.5,high
5,183.0,high
6,168.0,small


In [12]:
pd.get_dummies(df, drop_first = True, prefix='height', dtype=int)

,height,height_medium,height_high
0,175.0,0,0
1,178.5,1,0
2,185.0,0,1
3,191.0,0,1
4,184.5,0,1
5,183.0,0,1
6,168.0,0,0


Ekstrakcja cech

In [13]:
df = pd.DataFrame(data={'lang': [['PL', 'ENG'], ['GER', 'ENG', 'PL', 'FRA'], ['RUS']]})
df

,lang
0,"[PL, ENG]"
1,"[GER, ENG, PL, FRA]"
2,[RUS]


In [16]:
df['lang-number'] = df['lang'].apply(len)
df

,lang,lang-number
0,"[PL, ENG]",2
1,"[GER, ENG, PL, FRA]",4
2,[RUS],1


Sprawdzanie czy jest jezyk polski


In [17]:
df['PL_flag'] = df['lang'].apply(lambda x: 1 if 'PL' in x else 0)
#zwraca 1 gdy w x znajduje sie wartosc PL else 0
df

,lang,lang-number,PL_flag
0,"[PL, ENG]",2,1
1,"[GER, ENG, PL, FRA]",4,1
2,[RUS],1,0


In [18]:
df = pd.DataFrame(data={'website': ['wp.pl','onet.pl','google.com']})
df

,website
0,wp.pl
1,onet.pl
2,google.com


In [19]:
df.website.str.split('.', expand=True)

,0,1
0,wp,pl
1,onet,pl
2,google,com


In [21]:
new = df.website.str.split('.', expand=True)

df['name'] = new[0]
df['extension'] = new[1]
df

,website,name,extension
0,wp.pl,wp,pl
1,onet.pl,onet,pl
2,google.com,google,com
